In [2]:
# ====================================================
# LCELの基本
# ====================================================
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("system", "ユーザーが入力した料理のレシピを考えてください。"), ("human", "{dish}")]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

chain = prompt | model | output_parser

# invoke
output = chain.invoke({"dish": "ハンバーガー"})

# # stream（ストリーミング処理）
# for chunk in chain.stream({"dish": "ハンバーガー"}):
#     print(chunk, end="", flush=True)
# print(output)

# # batch（バッチ処理）
# outputs = chain.batch([{"dish": "ハンバーガー"}, {"dish": "チキンラーメン"}])
# print(outputs)


In [7]:
# ====================================================
# さまざまなRunnableを連鎖させる
# ====================================================
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

summary_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summary_chain = summary_prompt | model | output_parser

cot_summary_chain = cot_chain | summary_chain

output = cot_summary_chain.invoke({"question": "10 + 2 * 3 = ?"})

print(output)


結論: \( 10 + 2 \times 3 = 16 \) です。


In [10]:
# ====================================================
# 任意の関数をRunnableにする
# ====================================================
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([("system", "You are a helpful assistant."), ("human", "{input}")])

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()


def uppercase(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(uppercase)

output = chain.invoke({"input": "Hello, how are you?"})

print(output)


HELLO! I'M JUST A COMPUTER PROGRAM, SO I DON'T HAVE FEELINGS, BUT I'M HERE AND READY TO HELP YOU. HOW CAN I ASSIST YOU TODAY?


In [11]:
# RunnableLambdaの自動変換(そのまま連鎖させてあげると自動で変換される)
def uppercase(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | uppercase

output = chain.invoke({"input": "Hello, how are you?"})
print(output)


HELLO! I'M JUST A COMPUTER PROGRAM, SO I DON'T HAVE FEELINGS, BUT I'M HERE AND READY TO HELP YOU. HOW CAN I ASSIST YOU TODAY?


In [14]:
# 独自関数をstreamに対応させたい場合
from collections.abc import Iterator


def uppercase_stream(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | output_parser | uppercase_stream

for chunk in chain.stream({"input": "Hello, how are you?"}):
    print(chunk, end="", flush=True)


HELLO! I'M JUST A COMPUTER PROGRAM, SO I DON'T HAVE FEELINGS, BUT I'M HERE AND READY TO HELP YOU. HOW CAN I ASSIST YOU TODAY?

In [16]:
# ====================================================
# 複数のRunnableを並列につなげる
# ====================================================
import pprint

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは超楽観主義者です。ユーザーの入力に対して超楽観的な意見をください。"),
        ("human", "{input}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{input}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

pprint.pprint(parallel_chain.invoke({"input": "生成AIの進化について"}))


{'optimistic_opinion': '生成AIの進化は本当に素晴らしいですね！私たちの生活をより便利に、楽しくしてくれる可能性が無限大です。例えば、クリエイティブなプロジェクトやアイデアのインスピレーションを得る手助けをしてくれたり、教育や医療の分野でも革新をもたらしています。これからも進化を続けることで、私たちの想像を超えるような新しい体験や解決策が生まれるでしょう。未来は明るく、ワクワクすることがたくさん待っていますよ！',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものがありますが、その一方で多くの懸念も伴っています。技術が進化することで、私たちの仕事が奪われたり、情報の信頼性が低下したりするリスクが高まっています。さらに、AIが生成するコンテンツが人間の創造性を脅かし、私たちの思考や感情に悪影響を及ぼす可能性もあります。結局のところ、便利さの裏には常に不安がつきまとい、私たちの未来はますます不透明になっていくのではないでしょうか。'}


In [19]:
# RunnableParallelの出力をRunnableの入力に連結する
syntesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは至極客観的なAIです。2つの意見をまとめてください。"),
        ("human", "楽観的な意見: {optimistic_opinion}\n悲観的な意見: {pessimistic_opinion}"),
    ]
)

syntesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | syntesize_prompt
    | model
    | output_parser
)

for chunk in syntesize_chain.stream({"input": "生成AIの進化について"}):
    print(chunk, end="", flush=True)


生成AIの進化には、便利さや創造性の向上といった楽観的な側面がある一方で、仕事の喪失や情報の信頼性の低下といった悲観的な懸念も存在します。楽観的な意見では、生成AIが私たちの生活を豊かにし、新しいアイデアや視点を提供することで、より有意義な時間を過ごせる可能性が強調されています。しかし、悲観的な意見では、技術の進化がもたらすリスクや、AIが人間の創造性や思考に与える影響についての懸念が示されています。このように、生成AIの進化は、期待と不安が交錯する複雑な状況を生み出していると言えるでしょう。

In [20]:
# RunnableParallelへの自動変換(辞書を連結すると自動でRunnableParallelに変換される)
syntesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | syntesize_prompt
    | model
    | output_parser
)

for chunk in syntesize_chain.stream({"input": "生成AIの進化について"}):
    print(chunk, end="", flush=True)


生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な意見では、生成AIが私たちの生活を便利で楽しいものにし、創造性を引き出す新しいアイデアやアートを生み出す可能性が強調されています。教育、医療、エンターテインメントなどの分野での活用が進むことで、私たちの世界が豊かになることへの期待が表れています。

一方で、悲観的な意見では、生成AIの進化に伴うリスクが指摘されています。仕事の喪失や情報の信頼性の低下、AIによるコンテンツ生成が人間の創造性を脅かす可能性が懸念されています。便利さの裏には不安があり、未来が不透明になることへの警戒が示されています。

このように、生成AIの進化には多くの可能性と同時にリスクが存在し、私たちはその両面を理解しながら技術の進展に向き合う必要があります。

In [23]:
# itemgetterを使う例
from operator import itemgetter

# itemgetterの使い方を簡単に紹介
topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)


生成AIの進化について


In [26]:
# itemgetterを使ってpromptの穴埋めをする

syntesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは至極客観的なAIです。{input}について2つの意見をまとめてください。"),
        ("human", "楽観的な意見: {optimistic_opinion}\n悲観的な意見: {pessimistic_opinion}"),
    ]
)

syntesize_chain = (
    {
        "input": itemgetter("input"),  # 自動でRunnableLambdaに変換される
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | syntesize_prompt
    | model
    | output_parser
)

for chunk in syntesize_chain.stream({"input": "生成AIの進化について"}):
    print(chunk, end="", flush=True)


**楽観的な意見:** 生成AIの進化は、私たちの生活をより便利で楽しいものにする可能性を秘めています。クリエイティブなプロジェクトやアイデアのインスピレーションを得るための強力なツールとして活用でき、教育や医療の分野でも革新的な変化をもたらすでしょう。未来において、生成AIが私たちの想像を超える成果を生み出すことを期待し、新しい技術が進化することで生活が豊かになることを楽しみにしています。

**悲観的な意見:** 生成AIの進化には多くの懸念が伴います。技術の進化により、仕事が奪われたり、情報の信頼性が低下するリスクが高まります。また、AIが生成するコンテンツの氾濫により、オリジナリティやクリエイティビティが失われる可能性もあります。便利さの裏には常に代償があり、この技術をどのように扱うかが重要ですが、その管理がうまくいくかどうかは疑問です。

In [15]:
# ====================================================
# RunnablePassthrough-入力をそのまま出力する(RAGで良く使う)
# ====================================================
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''
    以下の文脈だけを踏まえて質問に回答してください。
    文脈："""
    {context}
    """

    質問：
    {question}
    ''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = TavilySearchAPIRetriever(
    k=5,
    search_depth="advanced",
    include_answer=True,
)

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

output = chain.invoke("2024年のM-1グランプリの優勝者は？")
print(output)


2024年のM-1グランプリの優勝者は「令和ロマン」です。彼らは史上初の連覇を達成しました。


In [18]:
# assignを使ってRunnableParallelの出力に値を追加する
# プロンプトの穴埋めした結果を画面に表示したい場合など、Chainの中間の値をUI上に表示したい場合等にassignが役立つ
import pprint

# from langchain_core.runnables import RunnableParallel

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

# # RunnableParallelのassignメソッドを使うと次のようにも書ける
# chain = RunnableParallel(
#     {
#         "question": RunnablePassthrough(),
#         "context": retriever,
#     }
# ).assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("2025年1月1日時点での日本の総理大臣は？")
pprint.pprint(output)


{'answer': '2025年1月1日時点での日本の総理大臣は石破茂氏です。',
 'context': [Document(metadata={'title': '2025年、石破首相に立ちはだかる3つの壁 36年前の悪夢とは…「ポスト石破」は誰？（2025年1月2日掲載）｜日テレnews Nnn', 'source': 'https://news.ntv.co.jp/category/politics/4731f536a40e4dec825d82372baabf19', 'score': 0.81875163, 'images': []}, page_content='2025年、石破首相に立ちはだかる3つの壁\u300036年前の悪夢とは…「ポスト石破」は誰？（2025年1月1日掲載）｜日テレNEWS NNN 日テレNEWS NNN 政治 2025年、石破首相に立ちはだかる3つの壁\u300036年前の悪夢とは…「ポスト石破」は誰？ 政治 2025年、石破首相に立ちはだかる3つの壁\u300036年前の悪夢とは…「ポスト石破」は誰？ ポスト 送る シェア ブックマーク URLをコピー 2025年1月1日 22:00 2024年10月1日、総理大臣に指名された石破茂氏。総裁選挙はなんと5度目の挑戦。 衆議院選挙で与党過半数割れに追い込まれた石破政権。2025年の通常国会では、最重要課題の予算成立に向け野党の協力を得るメドは立ってない。“12年に一度”の東京都議選と参院選がともに行われる年に、石破首相に立ちはだかる3つの壁とポスト石破について解説する。（政治部 自民党担当\u3000長谷栞里） ■第1の壁 2025年度予算\u3000“野党の協力”が必須 自民・公明が過半数の議席を得られず“少数与党”となり「国会の景色が一変した」永田町。衆議院予算委員会の委員長など、重要ポストが野党の手に渡る中、2025年は与党にとってさらに苦しい場面が続くことが予想される。   国会で審議される予算案や法案は基本的に衆議院・参議院で“過半数の賛成”で可決されて初めて成立する。つまり、現状の与党の衆議院の議席数では、野党にも賛成してもらわない限り、2025年度の本予算案が成立させられないのだ。   2024年、少数与党になって初めての臨時国会では「補正予算」を成立させるために